# GMC Issues Licence

Before issuing a licence the GMC makes a connection with the medic through a face to face meeting arranged by the medical school they are attending. It is here that personally identifiable information, including a headshot of the medic can be collected.

Before the GMC Licence is issued the GMC verifies the medic has recieved a Primary Medical Qualification from a medical school they trust.

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://general-medical-council-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = os.getenv("WEBHOOK_PORT")
webhook_host = "0.0.0.0"

agent_controller.init_webhook_server(webhook_host, webhook_port)
await agent_controller.listen_webhooks()

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Set Global Identifiers

These specify which schema and issuers to trust and would likely be defined by some governance framework.

In [4]:
pmq_schema_id = '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.1'

In [5]:
trusted_medical_school_dids = ["3jnXQcj9VLFjcUbtDVZZzV"] 


## Store Issuing Schema and Cred Def Identifiers

If you intend for this agent to issue credentials you should first initialise your agent as an issuer and author the relevant identifiers to the public ledger. The issuer_initialisation recipe notebook can be duplicated and used as a starting point.

Once schema and cred def identifiers are created copy across and store in variables as illustrated in the cell below. Be sure to use unique names for each variable.

In [6]:
gmc_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1'
gmc_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:12:default'
# <cred_def_id> = "<SOME CRED DEF ID>"

rtw_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:UK Right to Work:0.0.1'
rtw_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:18:doctor-onboarding'

dbs_schema_id = 'TDAbSf3Uqebg8N4XvybMbg:2:DBS Check:0.0.1'
dbs_cred_def_id = 'TDAbSf3Uqebg8N4XvybMbg:3:CL:19:doctor-onboarding'

%store gmc_schema_id
%store gmc_cred_def_id

%store rtw_schema_id
%store rtw_cred_def_id

%store dbs_schema_id
%store dbs_cred_def_id

Stored 'gmc_schema_id' (str)
Stored 'gmc_cred_def_id' (str)
Stored 'rtw_schema_id' (str)
Stored 'rtw_cred_def_id' (str)
Stored 'dbs_schema_id' (str)
Stored 'dbs_cred_def_id' (str)


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and fill in additional logic as needed.

In [7]:
listeners = []

In [8]:
## YOUR LISTENERS HERE
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")
    if state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [9]:
def issuer_handler(payload):
    
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    
    print(payload)
    
    print("\n---------------------------------------------------\n")

    
    if state == "offer_sent":
        proposal = payload["credential_proposal_dict"]
        attributes = proposal['credential_proposal']['attributes']

        print(f"Offering credential with attributes  : {attributes}")
        ## YOUR LOGIC HERE
    elif state == "request_received":
        print("Request for credential received")
        ## YOUR LOGIC HERE
    elif state == "credential_sent":
        print("Credential Sent")
        ## YOUR LOGIC HERE
    
issuer_listener = {
    "topic": "issue_credential",
    "handler": issuer_handler
}

listeners.append(issuer_listener)

In [10]:

def verifier_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    

    if state == "request_sent":
        print("Presentation Request\n")
        print(payload["presentation_request"])
        print("\nThe presentation request is encoded in base64 and packaged into a DIDComm Message\n")
        print(payload["presentation_request_dict"])
        print("\nNote the type defines the protocol present-proof and the message request-presentation\n")
        print(payload["presentation_request_dict"]["@type"])
    elif state == "presentation_received":
        print("Presentation Received")
        print("We will not go into detail on this payload as it is comparable to the presentation_sent we looked at in the earlier cell.")
        print("This is the full payload\n")
        print(payload)
    else:
        print("Paload \n")
        print(payload)
        
verifier_listener = {
    "topic": "present_proof",
    "handler": verifier_proof_handler
}

listeners.append(verifier_listener)


In [11]:
agent_controller.register_listeners(listeners)

Subscribing too: connections
Subscribing too: issue_credential
Subscribing too: present_proof


## Define Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections requesting that they produce a presentation containing the identified attributes and meeting the restrictions.

It is often useful to define your request objects first, then reuse these objects across many connections you wish to request a proof from. 

Duplicate and customise the below cell as many times as you need. It may be useful to save these request objects either to the jupyter store using %store or through

TODO: Detail the full set of restrictions available to a verifier.

In [12]:
import time

# Define the list of attributes and restrictions under which each attribute was issued that a prover must satisfy with a presentation
# NOTE: if identifying a schema or credential definition then the attribute name must be contained within the corresponding schema.
req_attrs = [
    {"name": "Name", "restrictions": [{"schema_id": pmq_schema_id}]},
    {"name": "University", "restrictions": [{"schema_id": pmq_schema_id}]},
    {"name": "Date Issued", "restrictions": [{"schema_id": pmq_schema_id}]},
]

# We could extend this to request the name attribute aswell if we wanted.


pmq_proof_request = {
    "name": "Proof of Primary Medical Qualification",
    "version": "1.0",
    "requested_attributes": {
        # They must follow this uuid pattern
        # Note that req_attr['name'] gets the attribute name of each object. E.g. domain and name in this case
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },

    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
    # You can also request the entire proof request be non-revoked
    "non_revoked":  {"to": int(time.time())}
}

## GMC and Medical Student Establish Connection

A GMC Administrative Staff would be introduced to each Medical student in a physical meeting organised by the medical school. As part of this meeting **which already happens**, the GMC Admin staff could establish a DIDComm connection between the ACA-Py agent representing the GMC (which they would have to be authorised to use) and the medic's agent.

Both parties can therefore have a high degree of confidence that the digital connection they just established represents the entity they think it does. E.g. a medical student, or the GMC.

In [13]:
# Alias for invited connection
alias = "Medic"
auto_accept = "true"
# Use public DID?
public = "false"
# Should this invitation be usable by multiple invitees?
multi_use = "false"

invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)
# Is equivalent to above. Arguments are optionally
# invitation_response = await agent_controller.connections.create_invitation()



# You will use this identifier to issue a credential across this connection
connection_id = invitation_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  a246422d-52ff-439b-9a35-6c32fa84ac13
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------


## Share Invitation Object with Medic

Again this would likely happen as part of a face to face interaction.

In [14]:
invitation = invitation_response["invitation"]
## Copy this output
print(invitation)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'd0e0c252-9582-4438-83bc-fc43d4f7ac39', 'label': 'General Medical Council', 'serviceEndpoint': 'https://107b751417ea.ngrok.io', 'recipientKeys': ['Hqo7q4HQRcokSj1b4UHVEQQBdLc9GNRGJNu1U53UWRbt']}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  a246422d-52ff-439b-9a35-6c32fa84ac13
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  a246422d-52ff-439b-9a35-6c32fa84ac13
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  a246422d-52ff-439b-9a35-6c32fa84ac13
State :  active
Routing State :  none
Th

## Optional: Display Invite as QR Code

Most likely the medic would be using a mobile to connect with the GMC

In [ ]:
import qrcode
# Link for connection invitation
invitation_url = invitation_response["invitation_url"]
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=5,
        border=5)
qr.add_data(invitation_url)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img

## Send Proof Request

This identifies a connection and a proof request object (you will need to update the variable name enclosed with <>) 

In [15]:
proof_request = {
    "comment": "Please prove that you have recieved a valid PMQ",
    "connection_id": connection_id,
    "proof_request": pmq_proof_request,
    # Do you want your agent to trace this request (for debugging)
    "trace": True
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  4faf558b-0769-4197-a928-860a4d2f1fc6
Presentation Exchange ID :  82f1aac1-0368-4b8b-89de-dd6ffe535900
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Primary Medical Qualification', 'version': '1.0', 'requested_attributes': {'0_Name_uuid': {'name': 'Name', 'restrictions': [{'schema_id': '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.1'}]}, '0_University_uuid': {'name': 'University', 'restrictions': [{'schema_id': '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.1'}]}, '0_Date Issued_uuid': {'name': 'Date Issued', 'restrictions': [{'schema_id': '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.1'}]}}, 'requested_predicates': {}, 'non_revoked': {'to': 1621339838}, 'nonce': '158153268944743895322245'}

The presen

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [16]:
presentation_exchange_id = proof_request_response["presentation_exchange_id"]

pres_record = await agent_controller.proofs.get_record_by_id(presentation_exchange_id)

print(pres_record)

{'updated_at': '2021-05-18 12:12:16.174710Z', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '89553b70-87fc-4e7c-81ef-29739ed8cd73', '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'Please prove that you have recieved a valid PMQ', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIFByaW1hcnkgTWVkaWNhbCBRdWFsaWZpY2F0aW9uIiwgInZlcnNpb24iOiAiMS4wIiwgInJlcXVlc3RlZF9hdHRyaWJ1dGVzIjogeyIwX05hbWVfdXVpZCI6IHsibmFtZSI6ICJOYW1lIiwgInJlc3RyaWN0aW9ucyI6IFt7InNjaGVtYV9pZCI6ICIzam5YUWNqOVZMRmpjVWJ0RFZaWnpWOjI6UHJpbWFyeSBNZWRpY2FsIFF1YWxpZmljYXRpb246MC4wLjEifV19LCAiMF9Vbml2ZXJzaXR5X3V1aWQiOiB7Im5hbWUiOiAiVW5pdmVyc2l0eSIsICJyZXN0cmljdGlvbnMiOiBbeyJzY2hlbWFfaWQiOiAiM2puWFFjajlWTEZqY1VidERWWlp6VjoyOlByaW1hcnkgTWVkaWNhbCBRdWFsaWZpY2F0aW9uOjAuMC4xIn1dfSwgIjBfRGF0ZSBJc3N1ZWRfdXVpZCI6IHsibmFtZSI6I

## Verify Presentation

Only if it is in the right state. 

Note: Verifying a presentation moves the state to `verified` regardless of whether the presentation request has been satisfied. To check this you must refer to the `verified` property on the response.

In [17]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

verified = verified_response["verified"]
print("Valid PMQ Presented : " + verified)

Valid PMQ Presented : true

---------------------------------------------------------------------

Handle present-proof
Connection ID :  4faf558b-0769-4197-a928-860a4d2f1fc6
Presentation Exchange ID :  82f1aac1-0368-4b8b-89de-dd6ffe535900
Protocol State :  verified
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Paload 

{'updated_at': '2021-05-18 12:13:36.014838Z', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '89553b70-87fc-4e7c-81ef-29739ed8cd73', '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'Please prove that you have recieved a valid PMQ', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIFByaW1hcnkgTWVkaWNhbCBRdWFsaWZpY2F0aW9uIiwgInZlcnNpb24iOiAiMS4wIiwgInJlcXVlc3RlZF9hdHRyaWJ1dGVzIjogeyIwX05hbWVfdXVpZ

## Fetching Issuing DID for Disclosed PMQ

The GMC will want to check that this credential was issued by a medical school they trust. Likely they would compare this against a list of verified medical schools.

Note: Other identifiers might also be useful for similar purposes.

In [24]:
identifiers_list = verified_response['presentation']["identifiers"]
print(identifiers_list)

for identifiers in identifiers_list:
    cred_def_id = identifiers["cred_def_id"]
    issuing_did = cred_def_id.split(":")[0]
    print(issuing_did)
    if issuing_did in trusted_medical_school_dids:
        print("Issuer is trusted to issued PMQ Credentials")
    

[{'schema_id': '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.1', 'cred_def_id': '3jnXQcj9VLFjcUbtDVZZzV:3:CL:10:default', 'rev_reg_id': None, 'timestamp': None}]
3jnXQcj9VLFjcUbtDVZZzV
Issuer is trusted to issued PMQ Credentials


## Parsing Disclosed Attribute Values from Presentation

A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

### Parse Revealed Attributes

In [25]:
for (name, val) in verified_response['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("Attribute : ", val)
    print("Raw Value : ", val['raw'])

Attribute :  {'sub_proof_index': 0, 'raw': 'Edinburgh Medical School', 'encoded': '57963125648994666300875361621729313351158154740138173103415020334594043694317'}
Raw Value :  Edinburgh Medical School
Attribute :  {'sub_proof_index': 0, 'raw': '2021-05-18', 'encoded': '43666059209831238854477176660850810175751782442498745015573788116669132641532'}
Raw Value :  2021-05-18
Attribute :  {'sub_proof_index': 0, 'raw': 'Will Abramson', 'encoded': '65005548905037764764502350053825769914874082648761230458670796241392891659611'}
Raw Value :  Will Abramson


## Issue GMC Licence

### Populate Credential Attributes

Before you can issue a credential, you must define the values that will be issued in this credential. The attribute names **MUST** match those in the schem identified by the <schema_id> value.


In [15]:
import base64
encoding = 'utf-8'
with open("test.txt", "r") as image:
    my_string =image.read()
#     print(my_string)
    headshotBase64 = my_string
    print(headshotBase64.replace("\n", ""))

adsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadssssssssssssssssssssssssssssssslllllllllllllll

In [16]:
from datetime import date, timedelta
import json

attributes = ["GMC Number", "Responsible Officer GMC Number", "Licenced From", "Re-Validation Due", "Name", "Headshot", "DOB"]

gmc_number="1231231" # input("Please enter the doctors GMC Number: ")
resp_officer="3827123"  # input("Please enter the GMC Number for the doctors responsible officer: ")
licenced_from=date.today().isoformat()
revalidation_due = (date.today() + timedelta(days=1825)).isoformat()
name="Will A"  # input("Please enter the doctors name")
headshot_holder=headshotBase64
dob=input("Enter doctors date of birth")
credential_attributes = [
    {"name": "GMC Number", "value": gmc_number},
    {"name": "Responsible Officer GMC Number", "value": resp_officer},
    {"name": "Licenced From", "value": licenced_from},
    {"name": "Re-Validation Due", "value": revalidation_due},
    {"name": "Name", "value": name},
    {"name": "Headshot", "value": headshot_holder},
    {"name": "DOB", "value": dob}
]
print(json.dumps(credential_attributes))

Enter doctors date of birth 2


[{"name": "GMC Number", "value": "1231231"}, {"name": "Responsible Officer GMC Number", "value": "3827123"}, {"name": "Licenced From", "value": "2021-05-18"}, {"name": "Re-Validation Due", "value": "2026-05-17"}, {"name": "Name", "value": "Will A"}, {"name": "Headshot", "value": "adsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadsssssssssssssssssssssssssssssssllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllddddddddddddddddddddddddddddddddddddsssssssssssssssssssssssssssssggggggggggggggggggggggggggggggggggggggadsss

## Issue GMC Licence Credential

GMC Licence is only issued once the GMC has verified the Primary Medical Qualification.


In [18]:
# Do you want the ACA-Py instance to trace it's processes (for testing/timing analysis)
trace = True
comment = ""
# Remove credential record after issued?
auto_remove = False

# Change <schema_id> and <cred_def_id> to correct pair. Cred_def_id must identify a definition to which your agent has corresponding private issuing key.
if verified == "true":
    print("Medic has completed their primary medical qualication")
    send_cred_response = await agent_controller.issuer.send_credential(connection_id, gmc_schema_id, gmc_cred_def_id, credential_attributes, comment, auto_remove, trace)
    gmc_licence_cred_ex_id = send_cred_response["credential_exchange_id"]
# Note last three args are optional.
# await agent_controller.issuer.send_credential(connection_id, <schema_id>, <cred_def_id, credential_attributes)

Medic has completed their primary medical qualication

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : a246422d-52ff-439b-9a35-6c32fa84ac13
Credential exchange ID : 669c1d8d-d4e7-42e4-8e1c-e175ddf00bbb
Agent Protocol Role :  issuer
Protocol State :  offer_sent

---------------------------------------------------

{'initiator': 'self', 'auto_issue': True, 'state': 'offer_sent', 'thread_id': 'b891ed41-bfcc-4071-8333-bcbffc5b4f4b', 'credential_offer': {'schema_id': 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1', 'cred_def_id': 'TDAbSf3Uqebg8N4XvybMbg:3:CL:12:default', 'key_correctness_proof': {'c': '47517720309318183480963585112080416749240054071586072900007837587553065479572', 'xz_cap': '10171793268731138864985367494011552770330894658163041148987442264971055845553869070203651716813446426849595183489733340855743089372341192808066052986844253175848375604215319648148129954853867577670493895261878082657432229241896624703994336420540448394184

In [ ]:
print(send_cred_response)

## Issue Right to Work Credential

Before anyone can receive a licence to practice from the GMC they must demonstrate they have the right to work in the UK.

They might be a UK Citizen, or have a correct VISA etc. The GMC administrative staff will need to verify their status as part of the process for onboarding new medical students.

This credential acts as evidence that those checks have taken place, which other healthcare institutions may be able to trust as opposed to having the recheck the paper documentation in every new organisation the doctor works throughout their career.

In [39]:
eligible = "true"

testHeadshot = "9j/4AAQSkZJRgABAQEASABIAAD/2wBDAMiKlq+Wfcivo6/h1cju////////////////////////////////////////////////////////////////////2wBDAdXh4f//////////////////////////////////////////////////////////////////////////////////wgARCADbAPoDASIAAhEBAxEB/8QAFwABAQEBAAAAAAAAAAAAAAAAAAECA//EABQBAQAAAAAAAAAAAAAAAAAAAAD/2gAMAwEAAhADEAAAAcAAAAAAALAAAAAAAACzWQAAAAC2aE2OSwAAAAAAAsAAAAAtJVNIMTeSAAAAAAAAAsAtM2ClKUlAlMTeAAAACgSwAAAsolUm1AAAMzeDIBSLACgARoZsLc0lg0lNpQAABLDE1BAsUgKBAtQsC5oiwqU3c6JLAQqwjWSpSSCwANLkgAFgWAtJrI1c7JNCKJQAmOmDIAABSALAADWs7MzQms6ABAsDI1NYMqGpTMsAALNZBSWUazTbNFABKIAomN8xZSyUNUxN5Io1jeSVTNACg1YKgqUAk0M464I1oxbDQGdQxKN2ZN4tMA0gssNWUiwAWCxkA0zoAXOgCAudZNSjDWRNQWUazoSiAUJNQzQLDUCalAECy5NMjUUysABSoKgoAMrCkKBZRLAD/8QAHxAAAgIBBAMAAAAAAAAAAAAAATARQCAQQVBgADFw/9oACAEBAAEFAuwH4dHCnoB4CbU4woOCj4Mw6bsONEPNEc0USiKBRsvbWEGtCioOjOVj08YRqMNsBaPXP//EABQRAQAAAAAAAAAAAAAAAAAAAHD/2gAIAQMBAT8BYP/EABQRAQAAAAAAAAAAAAAAAAAAAHD/2gAIAQIBAT8BYP/EABQQAQAAAAAAAAAAAAAAAAAAAJD/2gAIAQEABj8CYD//xAAiEAACAQQDAQADAQAAAAAAAAAAAREQICExMEBBUWBhgXH/2gAIAQEAAT8h/AHyoga6O3KqvoPPKqMfbgg0TR9uaaE6tSR1EpY7URRBmj3HUmLEhXvfWL9nguF2vy+axOqt/arPH9D5fOFcMBpkciZ5RrFqwKjcGR/aQQoowJSJ4JkY7EkdXse6TwokjME/CSR6IlECQjS9oQ3PHEmF+yTpI9kIj4JG6NSRjZP86Z0WHWODfo7EEHorYP8ACSSSTzZofMgaFa6NSZHLVHur0bQ7nu9PysXSTVKOTbgX0TwIVW8SrUIggddlV3Fh8UjUWExgirsNGrPKZgYuNKRUPLgjdjHumlXii3SSRml0k0kkVjQmnWDCoejLMjeBioyeYRC9ME4uIppVqjyz44PSOZVSkEdFU/d/rsaFbrg95vWPZ6j3oe0Vnl3/2gAMAwEAAgADAAAAEPPPPPPPNPPPPPPPPNPPPPPBPNPPPPPPPPPPPPMFBEPPPPPPPPPHOLHCMBFPPPPPPPPPMKPHPDIPOPPPLONCLHHPPPLNHGPIHIFENGFNKFGGHHPJPPLLNKHOPDLNPPPOPNPPALPPLNOIMCPPPPOKEFNLGNNLKLJHMPMIGLBHOPLMDFFPOLPFCBDBPJAOOCPKBPPFNLMPPJNIMFGOCLNLJCODDHLMDMPBMP/EABQRAQAAAAAAAAAAAAAAAAAAAHD/2gAIAQMBAT8QYP/EABQRAQAAAAAAAAAAAAAAAAAAAHD/2gAIAQIBAT8QYP/EACoQAAIBAgUCBwEBAQEAAAAAAAABESExEEFRYfCBwSAwcZGhseHRQPFQ/9oACAEBAAE/EP8AzciB/wCFYcTPmZCa6lYmR/4L/Ml5isRbwalNFDa3/wADynzFYknKVeWE5yLKCOaz/piSKx/whv0FC/6ar8+jq52E+cyFek1y7jX+OkZz8eCCbku5ZQSlVkl0OGSPrbYhVjn4Sopz12EpPP8Ag4eVH0OBJeRSBk5SSSXu8IFCbmIpxFGqVeglNdSt35sRTlElmRCIz+BU7ibnlZDUuhe3kKCl9dvAx7r7Fi6hmZni9iHSWilGxFRpRPk/R6ZFKxUha+OX2EpcEKuwUtt+wlPIaTUMm6O64vAlNy30EYu3Q1ipMSeghJvRYNEnLzPBUaejHNUUuu2CpMlSpEueMVvJsWjuLEUjqK9RRAnFi3gzoxurHacUq426DUJVmcW3m8o6YJnqQmk77ajZeTufDEHCqla2Y9She/gl9sU4UD9cWk4r8CIhkxCJ+IJslTUQ5Jzlp4KIaWw1frBNwmZpKmpV2f2Gm7qcp29BLmuajmQqwOCVasbgooJqmhJQoSutP6NKHTf8GUJJyhllTBYNR1hMVy9oxZUt0FSoJaFU23WGif6x1Hhcl+4jlONZGOlLtmJFoOo0VEt2Jt3dyyef9FboIIlE5jNEAr3roPUkrfvjRU/SQ34qQSRim2fR1nuJmLegmyijKpTS1LiitO5TYpMxudtGJ7qPCB3K2TZyNlZeNfykpcFCq1PrQkqT0VhKtPkRocT94QhIshKLYKMvBWVERmJEte0eCGOngd/AvAky0qVVcCRvciP6L6+FQzfuNxklXn8zErVaPeg0Wr9Klk29RVJUczbY7nbBEOkqCiJNSPLUaCz8Ly8MMaHlWjHalRVS6czEuXm4Ply/hbJXEoX2IS3sNJrG0kBRiShUXQVts18EOYsIajU9zYeyjLqXxeCQywWKeRFee5T1ZE5QZlDlZ4s5zcnOCPqIXHRSqsiVbbJerM6loG00JxYhutxJwRSfAsTWf8wQ6LBEw6jW28kEL3EWqVJmIxZkNUIN1kSpVfcggqeFcFcZUT6EKholLKHSpF4qQ9D7A1gkxg3MekfpFBvQWpGeprtoLTTuJ0oZiwnF0zHKicsU1Lb7MhXoSvUY2Qkjhur9RIrYqk/X4G4dML04mJKlCHBtGVLD7ceGbnX8M5WWDW/UqwTJJpz2/Rc/n74HRDUobZbUSnLcr274UHV7ClJZN9WhTFVGOViiGX+n2JJKg1Ij9Go4IIS+l/0kSm/USlTl7jUoVyqJ52/Sedv0VAkZJG2o2aUbjxcQ5ai24kWYmrnvYaQni0dyXPJ9SArbpCuJ/QmrrVewjqLthGr2uZokhSUKg67Frr9vBoQ9CCBKbma+RKKxSpf1IdBnaYJNQkilZDRsloUPpi0m09CCI4dLBppuEodyo9MiX0P4gdEafTQbmglWD6Gacun3i1Bthz8wQaS1k7dzKtxMo72FSk0HcdbCVU7VxmBulMXpW75XGSq9u/qK2/PnYVDa6nwFz8I3gjS6sJypx5zY5zY5zbBXkcN0VV3GvcS5/R1dB0Gekc+yeczKXLn9E5UjHGeRnz4xX1fRK1KPDMXVd/UZKd+e4685UWRpLIynmwm2l2z/ADDnNjnNjnNhLnMsUqnqOsNq8z2Euf3sXVPkSzb9+XJ5zMfOaitzkmQ6VHLe22fqSTssO19DUDcRZ/07e+DuOxoPMV2ZMVh3R/T++F3dBZdSxiqqiuO47BWHYd8f/9k="

date_completed = date.today().isoformat()
expiration_date = "N/A"

address = "Some Address"# input("Enter the current address")

credential_attributes = [
    {"name":"Eligible", "value": testHeadshot}, 
    {"name": "Date Completed","value":date_completed} , 
    {"name":"Expiration Date", "value": expiration_date}, 
    {"name":"Address","value":address}]
print(credential_attributes)

[{'name': 'Eligible', 'value': '9EAACAQQDAQADAQAAAAAAAAAAAREQICExMEBBUWBhgXH/2gAIAQEAAT8h/AHyoga6O3KqvoPPKqMfbgg0TR9uaaE6tSR1EpY7URRBmj3HUmLEhXvfWL9nguF2vy+axOqt/arPH9D5fOFcMBpkciZ5RrFqwKjcGR/aQQoowJSJ4JkY7EkdXse6TwokjME/CSR6IlECQjS9oQ3PHEmF+yTpI9kIj4JG6NSRjZP86Z0WHWODfo7EEHorYP8ACSSSTzZofMgaFa6NSZHLVHur0bQ7nu9PysXSTVKOTbgX0TwIVW8SrUIggddlV3Fh8UjUWExgirsNGrPKZgYuNKRUPLgjdjHumlXii3SSRml0k0kkVjQmnWDCoejLMjeBioyeYRC9ME4uIppVqjyz44PSOZVSkEdFU/d/rsaFbrg95vWPZ6j3oe0Vnl3/2gAMAwEAAgADAAAAEPPPPPPPNPPPPPPPPNPPPPPBPNPPPPPPPPPPPPMFBEPPPPPPPPPHOLHCMBFPPPPPPPPPMKPHPDIPOPPPLONCLHHPPPLNHGPIHIFENGFNKFGGHHPJPPLLNKHOPDLNPPPOPNPPALPPLNOIMCPPPPOKEFNLGNNLKLJHMPMIGLBHOPLMDFFPOLPFCBDBPJAOOCPKBPPFNLMPPJNIMFGOCLNLJCODDHLMDMPBMP/EABQRAQAAAAAAAAAAAAAAAAAAAHD/2gAIAQMBAT8QYP/EABQRAQAAAAAAAAAAAAAAAAAAAHD/2gAIAQIBAT8QYP/EACoQAAIBAgUCBwEBAQEAAAAAAAABESExEEFRYfCBwSAwcZGhseHRQPFQ/9oACAEBAAE/EP8AzciB/wCFYcTPmZCa6lYmR/4L/Ml5isRbwalNFDa3/wADynzFYknKVeWE5yLKCOaz/piSKx/whv0FC/6ar8+jq52E+cyFek1y7jX+OkZz8eCCbku5ZQSlVkl0OGSPrbY

In [40]:
# Do you want the ACA-Py instance to trace it's processes (for testing/timing analysis)
trace = True
comment = ""
# Remove credential record after issued?
auto_remove = False

send_rwt_cred_response = await agent_controller.issuer.send_credential(connection_id, rtw_schema_id, rtw_cred_def_id, credential_attributes, comment, auto_remove, trace)



---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 4faf558b-0769-4197-a928-860a4d2f1fc6
Credential exchange ID : f42b57c4-4b4a-4a9e-bcc8-a2c5523e953a
Agent Protocol Role :  issuer
Protocol State :  offer_sent

---------------------------------------------------

{'updated_at': '2021-05-18 12:40:42.827623Z', 'schema_id': 'TDAbSf3Uqebg8N4XvybMbg:2:UK Right to Work:0.0.1', 'trace': True, 'thread_id': '4e130e34-84ab-44c4-b9bf-c3783e463c6a', 'initiator': 'self', 'role': 'issuer', 'created_at': '2021-05-18 12:40:42.827623Z', 'credential_exchange_id': 'f42b57c4-4b4a-4a9e-bcc8-a2c5523e953a', 'auto_issue': True, 'credential_definition_id': 'TDAbSf3Uqebg8N4XvybMbg:3:CL:18:doctor-onboarding', 'credential_offer_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '4e130e34-84ab-44c4-b9bf-c3783e463c6a', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'c

## Issue DBS Check

Again the GMC is NOT the authority on DBS checks in the UK. However, they still might usefully be able to provide healthcare professionals with cryptographic evidence that these checks took place on a certain date and reference a certificate number that others can look up.



In [35]:
certificated_reference_number = input("Enter the DBS Certificate Reference Number")

date_completed = date.today().isoformat()
expiration_date = (date.today() + timedelta(days=1825)).isoformat()
dbs_check_type = headshotBase64 # input("Standard or Enhanced DBS Check")

credential_attributes = [
    {"name":"Certificate Reference Number", "value": certificated_reference_number}, 
    {"name": "Date Completed","value":date_completed} , 
    {"name":"Expiration Date", "value": expiration_date}, 
    {"name":"Type","value":dbs_check_type}]
print(credential_attributes)

Enter the DBS Certificate Reference Number 12


[{'name': 'Certificate Reference Number', 'value': '12'}, {'name': 'Date Completed', 'value': '2021-05-18'}, {'name': 'Expiration Date', 'value': '2026-05-17'}, {'name': 'Type', 'value': '/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAMiKlq+Wfcivo6/h1cju////////////////////////////////////////////////////////////////////2wBDAdXh4f//////////////////////////////////////////////////////////////////////////////////wgARCADbAPoDASIAAhEBAxEB/8QAFwABAQEBAAAAAAAAAAAAAAAAAAECA//EABQBAQAAAAAAAAAAAAAAAAAAAAD/2gAMAwEAAhADEAAAAcAAAAAAALAAAAAAAACzWQAAAAC2aE2OSwAAAAAAAsAAAAAtJVNIMTeSAAAAAAAAAsAtM2ClKUlAlMTeAAAACgSwAAAsolUm1AAAMzeDIBSLACgARoZsLc0lg0lNpQAABLDE1BAsUgKBAtQsC5oiwqU3c6JLAQqwjWSpSSCwANLkgAFgWAtJrI1c7JNCKJQAmOmDIAABSALAADWs7MzQms6ABAsDI1NYMqGpTMsAALNZBSWUazTbNFABKIAomN8xZSyUNUxN5Io1jeSVTNACg1YKgqUAk0M464I1oxbDQGdQxKN2ZN4tMA0gssNWUiwAWCxkA0zoAXOgCAudZNSjDWRNQWUazoSiAUJNQzQLDUCalAECy5NMjUUysABSoKgoAMrCkKBZRLAD/8QAHxAAAgIBBAMAAAAAAAAAAAAAATARQCAQQVBgADFw/9oACAEBAAEFAuwH4dHCnoB4CbU4woOCj4Mw6bsONEPNEc0USiKBRsvb

In [36]:
# Do you want the ACA-Py instance to trace it's processes (for testing/timing analysis)
trace = True
comment = ""
# Remove credential record after issued?
auto_remove = False

send_dbs_cred_response = await agent_controller.issuer.send_credential(connection_id, dbs_schema_id, dbs_cred_def_id, credential_attributes, comment, auto_remove, trace)



---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 4faf558b-0769-4197-a928-860a4d2f1fc6
Credential exchange ID : 6035df65-56cc-4441-9137-ba08e1d21c37
Agent Protocol Role :  issuer
Protocol State :  offer_sent

---------------------------------------------------

{'updated_at': '2021-05-18 12:28:04.211593Z', 'schema_id': 'TDAbSf3Uqebg8N4XvybMbg:2:DBS Check:0.0.1', 'trace': True, 'thread_id': '11c7dbfb-efc8-43e6-93aa-f7c446e7fc29', 'initiator': 'self', 'role': 'issuer', 'created_at': '2021-05-18 12:28:04.211593Z', 'credential_exchange_id': '6035df65-56cc-4441-9137-ba08e1d21c37', 'auto_issue': True, 'credential_definition_id': 'TDAbSf3Uqebg8N4XvybMbg:3:CL:19:doctor-onboarding', 'credential_offer_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '11c7dbfb-efc8-43e6-93aa-f7c446e7fc29', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'create a

## Optional: Revoke GMC Licence Credential

Only possible if the credential issued used a revocable credential definition. (see support_revocation flag in write_cred_def() function).

There are two approachs to revoke an revocable credential

### Option 1: Use credential Exchange ID

Note: In a SSI application as an issuer you are might want to keep track of the identifiers for credential echange records. These can be used to retrieve records stored by the agent and also be used to identify a credential to revoke
Note though that applications might not wish to retain these records as they contain sensitive PII

In [32]:
response = await agent_controller.issuer.get_records()
record = response["results"][0]
print(record)

{'updated_at': '2021-05-18 12:23:26.835769Z', 'schema_id': 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1', 'trace': True, 'revoc_reg_id': 'TDAbSf3Uqebg8N4XvybMbg:4:TDAbSf3Uqebg8N4XvybMbg:3:CL:12:default:CL_ACCUM:397b8acd-bac7-42ca-bb0e-502a432300b5', 'credential': {'schema_id': 'TDAbSf3Uqebg8N4XvybMbg:2:GMC Licence:0.0.1', 'cred_def_id': 'TDAbSf3Uqebg8N4XvybMbg:3:CL:12:default', 'rev_reg_id': 'TDAbSf3Uqebg8N4XvybMbg:4:TDAbSf3Uqebg8N4XvybMbg:3:CL:12:default:CL_ACCUM:397b8acd-bac7-42ca-bb0e-502a432300b5', 'values': {'Licenced From': {'raw': '2021-05-18', 'encoded': '43666059209831238854477176660850810175751782442498745015573788116669132641532'}, 'Re-Validation Due': {'raw': '2026-05-17', 'encoded': '26886328372578875105311049009020895128863716097806395257395810936222704116782'}, 'Responsible Officer GMC Number': {'raw': '3827123', 'encoded': '3827123'}, 'Headshot': {'raw': '/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAMiKlq+Wfcivo6/h1cju/////////////////////////////////////////////////////////////////

In [33]:
record_id = record["credential_exchange_id"]
try:
    is_record = await agent_controller.issuer.get_record_by_id(record_id)
except:
    print("Record not found. Did your agent automatically remove it?")

if record:
    # For revoke_credential() you only need to provide (cred_ex_id) OR (rev_reg_id AND cred_rev_id).
    response = await agent_controller.revocations.revoke_credential(record_id, publish=True)
    response

### Option 2: Remove Credential using the Registry and Credential Revocation ID

Does not currently work because of bug in code

In [ ]:
registry = await agent_controller.revocations.get_active_revocation_registry_by_cred_def(cred_def_id)
print(registry)

In [ ]:
revoc_reg_id = registry["result"]["revoc_reg_id"]

In [ ]:
# Public revocation registry update to ledger
publish = True
# Believe this is index in rev registry starting from 1
cred_rev_id = 1
response = await agent_controller.revocations.revoke_credential(cred_rev_id = cred_rev_id, rev_reg_id = revoc_reg_id, publish=True)

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()